# Motivation #1 - Provide faster access to non-cloud-optimized GeoTIFFS

In [2]:
import xarray as xr
import rioxarray
import zarr

import os

# ref: https://developmentseed.org/titiler/advanced/performance_tuning/
os.environ["GDAL_DISABLE_READDIR_ON_OPEN"] = "EMPTY_DIR"

## Time performance using S3 protocol

Time opening the dataset with GDAL (via rioxarray and rasterio)

In [2]:
%%time
s3_uri = "s3://nasa-veda-scratch/geotiff-exploration/wbgtmax.2006.01.10.tif"
ds = rioxarray.open_rasterio(s3_uri, chunks={})

CPU times: user 78.1 ms, sys: 23.8 ms, total: 102 ms
Wall time: 374 ms


Time loading all the data

In [3]:
%%time
ds.load()

CPU times: user 317 ms, sys: 111 ms, total: 429 ms
Wall time: 2.19 s


<xarray.DataArray (band: 1, y: 2600, x: 7200)> Size: 75MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 58kB -180.0 -179.9 -179.9 ... 179.9 179.9 180.0
  * y            (y) float64 21kB 69.97 69.92 69.87 ... -59.88 -59.93 -59.98
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

## Predict performance of GeoTIFF virtualization as Zarr

The Zarr-Python reader on a virtualized GeoTIFF should yield the same performance as reading a Zarr dataset with the chunking scheme. Let's test how fast that would open and load.

Convert the dataset to a Zarr store with the same chunking scheme

In [4]:
output_store = "s3://nasa-veda-scratch/geotiff-exploration/wbgtmax.2006.01.10.zarr"

In [5]:
write_data = False

In [6]:
if write_data:
    ds = rioxarray.open_rasterio(s3_uri, chunks={})
    ds.to_zarr(output_store, mode="w", consolidated=True)

Time opening the dataset using Zarr

In [7]:
%%time
ds = xr.open_zarr(output_store, chunks={})
ds

CPU times: user 310 ms, sys: 36.7 ms, total: 346 ms
Wall time: 605 ms


<xarray.Dataset> Size: 75MB
Dimensions:                        (band: 1, y: 2600, x: 7200)
Coordinates:
  * band                           (band) int64 8B 1
  * x                              (x) float64 58kB -180.0 -179.9 ... 180.0
  * y                              (y) float64 21kB 69.97 69.92 ... -59.98
Data variables:
    __xarray_dataarray_variable__  (band, y, x) float32 75MB dask.array<chunksize=(1, 2600, 7200), meta=np.ndarray>
    spatial_ref                    int64 8B ...

Time loading all the data

In [8]:
%%time
ds.load()

CPU times: user 110 ms, sys: 102 ms, total: 212 ms
Wall time: 520 ms


<xarray.Dataset> Size: 75MB
Dimensions:                        (band: 1, y: 2600, x: 7200)
Coordinates:
  * band                           (band) int64 8B 1
  * x                              (x) float64 58kB -180.0 -179.9 ... 180.0
  * y                              (y) float64 21kB 69.97 69.92 ... -59.98
Data variables:
    __xarray_dataarray_variable__  (band, y, x) float32 75MB nan nan ... nan nan
    spatial_ref                    int64 8B 0

Actually, for a true apples-to-apples comparison we need to drop the coordinates because these should be inferred from the metadata (see for details - https://discourse.pangeo.io/t/example-which-highlights-the-limitations-of-netcdf-style-coordinates-for-large-geospatial-rasters/4140)

In [9]:
output_store = (
    "s3://nasa-veda-scratch/geotiff-exploration/wbgtmax.2006.01.10-no-coords.zarr"
)

In [10]:
if write_data:
    ds = rioxarray.open_rasterio(s3_uri, chunks={})
    ds = ds.drop_vars(["band", "x", "y"])
    ds.to_zarr(output_store, mode="w", consolidated=True)

Time opening the dataset using Zarr

In [11]:
%%time
ds = xr.open_zarr(output_store, chunks={})
ds

CPU times: user 11.4 ms, sys: 577 μs, total: 12 ms
Wall time: 24.3 ms


<xarray.Dataset> Size: 75MB
Dimensions:                        (band: 1, y: 2600, x: 7200)
Dimensions without coordinates: band, y, x
Data variables:
    __xarray_dataarray_variable__  (band, y, x) float32 75MB dask.array<chunksize=(1, 2600, 7200), meta=np.ndarray>
    spatial_ref                    int64 8B ...

Time loading all the data

In [12]:
%%time
ds.load()

CPU times: user 111 ms, sys: 100 ms, total: 211 ms
Wall time: 482 ms


<xarray.Dataset> Size: 75MB
Dimensions:                        (band: 1, y: 2600, x: 7200)
Dimensions without coordinates: band, y, x
Data variables:
    __xarray_dataarray_variable__  (band, y, x) float32 75MB nan nan ... nan nan
    spatial_ref                    int64 8B 0

Try it without xarray, just loading the data from Zarr

In [13]:
%%time
store = zarr.storage.FsspecStore.from_url(
    output_store,
    read_only=True,
)
group = zarr.open_group(store=store, mode="r")

CPU times: user 95.7 ms, sys: 12.7 ms, total: 108 ms
Wall time: 222 ms


In [14]:
%%time
group["__xarray_dataarray_variable__"][:]

CPU times: user 104 ms, sys: 57.4 ms, total: 162 ms
Wall time: 314 ms


array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)